# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [5]:
!pip install pymongo

In [6]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [7]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [8]:
# confirm that our new database was created
mongo.list_database_names()

['admin', 'config', 'local', 'met', 'uk_food']

In [9]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [10]:
# review the collections in our new database
collections = db.list_collection_names() 

In [11]:
# review the collections in our new database
collections = db.list_collection_names()

In [12]:
# review a document in the establishments collection
establishment = db.establishments.find_one()

In [13]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [78]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "business_name": "Penang Flavours",
    "business_type": "Restaurant",
    "address": {
        "street": "123 Greenwich High Road",
        "city": "Greenwich",
        "postcode": "SE10 8JA"
    },
    "phone_number": "+44 20 7946 0627",
    "rating": None,
    "cuisine_type": "Halal"
}

In [17]:
# Insert the new restaurant into the collection
db.establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('677cb7782b2b04896342aa43'), acknowledged=True)

In [18]:
# Check that the new restaurant was inserted
new_restaurant = db.establishments.find_one({"business_name": "Penang Flavours"})

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [20]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = db.establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [22]:
# Define the BusinessTypeID 
business_type_id = 1

In [23]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one(
    {"Business_name": "Penang Flavours"},
    {"$set": {"BusinessTypeID": business_type_id}}
)

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [24]:
# Confirm that the new restaurant was updated
updated_restaurant = db.establishments.find_one({"business_name": "Penang Flavours"})

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [26]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = db.establishments.count_documents({"LocalAuthorityName": "Dover"})

In [27]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = db.establishments.delete_many({"LocalAuthorityName": "Dover"})

In [28]:
# Check if any remaining documents include Dover
remaining_dover_docs = db.establishments.count_documents({"LocalAuthorityName": "Dover"})

In [29]:
# Check that other documents remain with 'find_one'
remaining_document = db.establishments.find_one()

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [32]:
# Change the data type from String to Decimal for longitude and latitude

db.establishments.update_many(
    {"longitude": {"$type": "string"}, "latitude": {"$type": "string"}},
    {"$set": {
        "longitude": {"$convert": {"input": "$longitude", "to": "decimal"}},
        "latitude": {"$convert": {"input": "$latitude", "to": "decimal"}}
    }}
)

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [34]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [35]:
# Change the data type from String to Integer for RatingValue

cursor = db.establishments.find({"RatingValue": {"$type": "string"}})
for document in cursor:
    rating_value_str = document["RatingValue"]
    rating_value_int = convert_to_int(rating_value_str)
    if rating_value_int is not None:
        db.establishments.update_one(
            {"_id": document["_id"]},
            {"$set": {"RatingValue": rating_value_int}}
        )

In [36]:
# Check that the coordinates and rating value are now numbers

sample_document = db.establishments.find_one(
    {"RatingValue": {"$type": "int"}, "longitude": {"$type": "decimal"}, "latitude": {"$type": "decimal"}},
    {"RatingValue": 1, "longitude": 1, "latitude": 1, "_id": 0}
)

In [37]:
# Create a dictionary for the new restaurant
new_restaurant = {
    "business_name": "Penang Flavours",
    "business_type": "Restaurant",
    "address": {
        "street": "123 Greenwich High Road",
        "city": "Greenwich",
        "postcode": "SE10 8JA"
    },
    "phone_number": "+44 20 7946 0627",
    "rating": None,
    "cuisine_type": "Halal"
}

In [38]:
# Find the business type ID for the restaurant/cafe/canteen and return only the business type IDS and business type fields.
business_type = db.establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)


In [39]:
# Update the new restaurant with the correct BusinessTypeID
if business_type:
    db.establishments.update_one(
        {"business_name": "Penang Flavours"},
        {"$set": {"BusinessTypeID": business_type_id}}
    )
    print("New restaurant updated with BusinessTypeID!")
else:
    print("BusinessTypeID for 'Restaurant/Cafe/Canteen' not found.")


BusinessTypeID for 'Restaurant/Cafe/Canteen' not found.


In [40]:
# Find how many documents contain the Dover Local Authority
dover_count = db.establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName as 'Dover': {dover_count}")

Number of documents with LocalAuthorityName as 'Dover': 0


In [41]:
# Remove any establishments within the Dover Local Authority
delete_result = db.establishments.delete_many({"LocalAuthorityName": "Dover"})
print(f"Number of documents deleted: {delete_result.deleted_count}")

Number of documents deleted: 0


In [42]:
# Find the number of documents to ensure they were deleted
remaining_docs_count = db.establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Remaining documents with LocalAuthorityName as 'Dover': {remaining_docs_count}")

Remaining documents with LocalAuthorityName as 'Dover': 0


In [43]:
# Update latitude and longitude fields from String to Decimal128
db.establishments.update_many(
    {"latitude": {"$type": "string"}, "longitude": {"$type": "string"}},
    [{"$set": {"latitude": {"$toDecimal": "$latitude"}, "longitude": {"$toDecimal": "$longitude"}}}]
)

print("Latitude and longitude fields updated to Decimal128.")


Latitude and longitude fields updated to Decimal128.


In [44]:
# Update RatingValue field from String to Integer
cursor = db.establishments.find({"RatingValue": {"$type": "string"}})
for document in cursor:
    rating_value_str = document["RatingValue"]
    rating_value_int = convert_to_int(rating_value_str)
    if rating_value_int is not None:
        db.establishments.update_one(
            {"_id": document["_id"]},
            {"$set": {"RatingValue": rating_value_int}}
        )

print("RatingValue fields updated to integer.")


RatingValue fields updated to integer.
